# Evaluation

In [ ]:
from eval.common import evaluate, evaluate_mAP
from generators.pascal import PascalVocGenerator
from model import efficientdet
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

### Definitions:

In [ ]:
phi = 0
weighted_bifpn = False
dataset_path = '/datasets/dataset/VOCdevkit/VOC2007'
model_path = 'checkpoints/voc_ciou_finetuned/pascal.h5'

### Make evaluation:

In [ ]:
common_args = {
    'batch_size': 1,
    'phi': phi,
}
test_generator = PascalVocGenerator(
    dataset_path,
    'test',
    shuffle_groups=False,
    skip_truncated=False,
    skip_difficult=True,
    **common_args
)

input_shape = (test_generator.image_size, test_generator.image_size)
anchors = test_generator.anchors
num_classes = test_generator.num_classes()
model, prediction_model = efficientdet(phi=phi, num_classes=num_classes, weighted_bifpn=weighted_bifpn)
prediction_model.load_weights(model_path, by_name=True)

#### IoU eval:

In [ ]:
average_precisions = evaluate_mAP(test_generator, prediction_model, method='iou', visualize=False)

# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations), test_generator.label_to_name(label),
          'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)
mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('mAP0.5:0.95 = {:.4f}'.format(mean_ap))

#### ProbIoU eval:

In [ ]:
average_precisions = evaluate_mAP(test_generator, prediction_model, method='probiou', visualize=False)

# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations), test_generator.label_to_name(label),
          'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)
mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('mAP0.5:0.95 = {:.4f}'.format(mean_ap))